In [1]:
from cobra import io
from cobra.core import Metabolite
from cobra import flux_analysis
import pandas as pd
from Functions import *
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "./../plantcoremetabolism-model/PlantCoreMetabolism_v2_1_0.xml"
model = io.sbml.read_sbml_model(fname)
display.clear_output()

#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
rxn.add_metabolites({met:1})


In [2]:
#remove charged state of PGPs since they are minor (0.03)
model = removeSpecificMetChargedState(model,["aL_1_PHOSPHATIDYL_GLYCEROL_P_p","aL_1_PHOSPHATIDYL_GLYCEROL_P_m"])

In [3]:


biomass = createEmptyBiomassDataFrame()
biomass

,type,leaf,stem,root,seed
,,,,,
sSUCROSE_b,,0.0,0.0,0.0,0.0
GLC_c,,0.0,0.0,0.0,0.0
FRU_c,,0.0,0.0,0.0,0.0
Starch_b,,0.0,0.0,0.0,0.0
Cellulose_b,,0.0,0.0,0.0,0.0
...,...,...,...,...,...
pLEU_b,protein,0.0,0.0,0.0,0.0
pPRO_b,protein,0.0,0.0,0.0,0.0
pCYS_b,protein,0.0,0.0,0.0,0.0


In [4]:
proteinMW = model.metabolites.get_by_id("Protein_b").formula_weight

C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2182716051S0.1420323181)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 32.8863668319 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2182716051S0.1420323181)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 4.9536245179 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2182716051S0.1420323181)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 9.2182716051 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2182716051S0.1420323181)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2

### Mature leaves

Extract day-time starch:sucrose accumulation rate ratio from Lunn and Hatch 1995, Figure 1

<img src="references/Screenshot_20190714_104104.png" height=25% width=25%>

In [5]:
df_matureleaf_content_1 = pd.DataFrame(data={"Time":["Dawn","Dusk"],
                                             "Hexose (umol hexose/mgChl)":[24.126984126984112-22.222222222222204,
                                                                           121.26984126984127-116.19047619047619],
                                             "Sucrose (umol hexose/mgChl)":[22.222222222222204-17.142857142857146,
                                                                            116.19047619047619-95.87301587301586],
                                             "Starch (umol hexose/mgChl)":[17.142857142857146,95.87301587301586]})

In [6]:
Delta_sucrose = float(df_matureleaf_content_1["Sucrose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk"]) - \
float(df_matureleaf_content_1["Sucrose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn"])

Delta_hexose =float(df_matureleaf_content_1["Hexose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk"]) - \
float(df_matureleaf_content_1["Hexose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn"])

Delta_starch = float(df_matureleaf_content_1["Starch (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk"]) - \
float(df_matureleaf_content_1["Starch (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn"])

df_matureleaf_content_1["Starch (dusk-dawn)"]=["-",Delta_starch]
df_matureleaf_content_1["Sucrose (dusk-dawn)"]=["-",Delta_sucrose]
df_matureleaf_content_1["Hexose (dusk-dawn)"]=["-",Delta_hexose]
df_matureleaf_content_1

,Time,Hexose (umol hexose/mgChl),Sucrose (umol hexose/mgChl),Starch (umol hexose/mgChl),Starch (dusk-dawn),Sucrose (dusk-dawn),Hexose (dusk-dawn)
0,Dawn,1.904762,5.079365,17.142857,-,-,-
1,Dusk,5.079365,20.317460,95.873016,78.7302,15.2381,3.1746


In [7]:
Sucrose_in_umols = 0.5*(df_matureleaf_content_1.at[1,"Sucrose (dusk-dawn)"])
Starch_in_umols = (df_matureleaf_content_1.at[1,"Starch (dusk-dawn)"])
Starch_to_sucrose_ratio = Starch_in_umols/Sucrose_in_umols
print ("Starch sucrose day-time accumulation rate = "+str(Starch_to_sucrose_ratio))

Starch sucrose day-time accumulation rate = 10.333333333333307


### Leaf

Extract major sugar and amino acid content (8hr into photoperiod) from Ogbaga et al 2016, Figure 3, 4 and 5

<img src="references/Screenshot_20190715_102019.png" height=30% width=30%>

<img src="references/Screenshot_20190715_101103.png" height=50% width=50%>

<img src="references/Screenshot_20190715_102807.png" height=50% width=50%>

In [8]:
df_leaf_content_1 = pd.DataFrame(data={"Metabolite (mg/gDW)":["sucrose","glucose","fructose",
                                                              "ASP","SER",
                                                              "THR","ASN",
                                                              "PRO","PHE",
                                                              "TYR","TRP"],
                                       "variety Samsorg 17":[7.43944636678199,40.2479338842975,0.401372212692967,
                                                             0.0455882352941172,0.0260162601625953,
                                                             0.18373983739837,0.864705882352938,
                                                             0,0.00977272727272728,
                                                             0.0324175824175804,0],
                                       "variery Samsorg 40":[3.42560553633216,38.3471074380165,0,
                                                             0.0553921568627444,0.0357723577235695,
                                                             0.403252032520321,1.54705882352941,
                                                             0,0.0188636363636364,
                                                             0.0357142857142827,0]})
df_leaf_content_1.set_index("Metabolite (mg/gDW)",drop=True,inplace=True)
df_leaf_content_1

,variety Samsorg 17,variery Samsorg 40
Metabolite (mg/gDW),,
sucrose,7.439446,3.425606
glucose,40.247934,38.347107
fructose,0.401372,0.000000
ASP,0.045588,0.055392
SER,0.026016,0.035772
THR,0.183740,0.403252
ASN,0.864706,1.547059
PRO,0.000000,0.000000
PHE,0.009773,0.018864


Gathered soluble sugar, starch, cellulose, hemicellulose and lignin content from Wang et al 2013, Table 2

In [9]:
df_leaf_content_2 = pd.DataFrame(data={"Soluble sugar (mg/gDW)":[69.3],"Starch (mg/gDW)":[10.7],
                                       "Cellulose (mg/gDW)":[247.6],"Hemicellulose (mg/gDW)":[219.7],
                                       "Lignin (mg/gDW)":[13.2]})
df_leaf_content_2

,Soluble sugar (mg/gDW),Starch (mg/gDW),Cellulose (mg/gDW),Hemicellulose (mg/gDW),Lignin (mg/gDW)
0,69.3,10.7,247.6,219.7,13.2


Extracted phospholipid content from Vanhercke et al 2019 Figure 4

<img src="references/Screenshot_20190715_130438.png" height=100% width=100%>

In [10]:
df_leaf_content_3 = pd.DataFrame(data={"DAG (mmol/gDW)":[4.71830985915491],"DGDG (mmol/gDW)":[14.2253521126762],
                                       "LPC (mmol/gDW)":[0.985915492957744],"MGDG (mmol/gDW)":[13.8732394366197],
                                       "PC (mmol/gDW)":[4.71830985915491],"TAG (mmol/gDW)":[0.845070422535294]})

df_leaf_content_3

,DAG (mmol/gDW),DGDG (mmol/gDW),LPC (mmol/gDW),MGDG (mmol/gDW),PC (mmol/gDW),TAG (mmol/gDW)
0,4.71831,14.225352,0.985915,13.873239,4.71831,0.84507


In [11]:
df_leaf_content_5_pre = pd.DataFrame(data={"TAG fatty acid":["C48:0","C48:1","C48:2","C48:3","C50:0",
                                                         "C50:1","C50:2","C50:3","C52:1","C52:2",
                                                         "C52:3","C52:4","C52:5","C52:6","C54:1",
                                                         "C54:2","C54:3","C54:4","C54:5","C54:6",
                                                         "C54:7","C54:8","C54:9","C56:1","C56:2",
                                                         "C56:3","C56:4","C56:5","C56:6"],
                                       "nmol/mgDW":[0.55126791620728,0.396912899669241,0.066152149944874,
                                                       0.022050716648291,0.110253583241457,1.12458654906285,
                                                       7.1664829106946,2.80044101433297,0.286659316427784,
                                                       1.30099228224917,1.56560088202867,4.27783902976847,
                                                       3.63836824696803,0.77177508269019,0.110253583241457,
                                                       0.507166482910698,0.242557883131201,0.573318632855572,
                                                       0.529217199558989,0.463065049614115,0.374862183020949,
                                                       0.15435501653804,0.044101433296583,0.066152149944874,
                                                       0.264608599779492,0.15435501653804,0.264608599779497,
                                                       0.242557883131205,0.044101433296583],
                                       "C16:0":[3,2,1,0,2,2,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                       "C16:1":[0,1,2,3,0,0,1,2,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                       "C16:2":[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                       "C16:3":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                       "C18:0":[0,0,0,0,1,0,0,0,1,0,0,0,0,0,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0],
                                       "C18:1":[0,0,0,0,0,1,1,1,1,2,2,1,0,0,1,2,3,2,1,0,0,0,0,1,2,2,1,0,0],
                                       "C18:2":[0,0,0,0,0,0,0,0,0,0,0,1,2,2,0,0,0,1,2,3,2,1,0,0,0,0,1,2,1],
                                       "C18:3":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,0,0,0,0,1],
                                       "C20:0":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0],
                                       "C20:1":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1]})
df_leaf_content_5_proc = pd.DataFrame(data={"":df_leaf_content_5_pre.columns[2:]})
tempList = list()
for cols in df_leaf_content_5_pre.columns[2:]:
    tot = 0
    for i in range(0,len(df_leaf_content_5_pre)):
        tot = tot + (df_leaf_content_5_pre[cols][i]*df_leaf_content_5_pre["nmol/mgDW"][i])
    tempList.append(tot)
df_leaf_content_5_proc["nmol/mgDW"]=tempList
df_leaf_content_5_proc.set_index("",drop=True,inplace=True)
df_leaf_content_5_proc

,nmol/mgDW
,
C16:0,13.737596
C16:1,22.844542
C16:2,0.771775
C16:3,0.000000
C18:0,1.190739
C18:1,26.085998
C18:2,17.816979
C18:3,0.859978
C20:0,0.330761


Estimate starch content using starch:sucrose ratio at dusk (from Lunn and Hatch 1995) and sucrose content (from Ogbaga et al 2016)

In [12]:
starch_mass = df_matureleaf_content_1["Starch (umol hexose/mgChl)"][1]* \
                                                    model.metabolites.get_by_id("GLC_c").formula_weight
sucrose_mass = df_matureleaf_content_1["Sucrose (umol hexose/mgChl)"][1]* \
                                                    model.metabolites.get_by_id("GLC_c").formula_weight

starch_mass2 = df_leaf_content_2["Starch (mg/gDW)"][0]
sucrose_mass2 = df_leaf_content_2["Soluble sugar (mg/gDW)"][0]
    

df_leaf_content_4 = pd.DataFrame(data={"Starch from Lunn and Hatch 1995 & Ogbaga et al 2016":
                                       [find_average(df_leaf_content_1.loc["sucrose"])*starch_mass/sucrose_mass],
                                       "Starch from Wang et al 2013 & Ogbaga et al 2016":
                                       [(find_average(df_leaf_content_1.loc["sucrose"])+
                                         find_average(df_leaf_content_1.loc["glucose"])+
                                         find_average(df_leaf_content_1.loc["fructose"]))*starch_mass2/sucrose_mass2]})
df_leaf_content_4

,Starch from Lunn and Hatch 1995 & Ogbaga et al 2016,Starch from Wang et al 2013 & Ogbaga et al 2016
0,25.634732,6.937357


### Summary

- Soluble sugar and amino acid data from Ogbaga et al 2016
- Phospholipid content was gathered from Vanhercke et al 2019

Assumptions

- Soluble sucrose content data from Ogbaga et al 2016 and Wang et al 2013 are very different. Values from the latter study was used to scale down, cellulose, hemicellulose and lignin
- Starch content predicted from Lunn and Hatch 1995 & Ogbaga et al 2016 is drastically different from that predicted from Starch from Wang et al 2013 & Ogbaga et al 2016. Values from the former was used for biomass.

In [13]:
biomass.at["COUMARATE_c","leaf"]

0.0

In [14]:
biomass.at["sSUCROSE_b","leaf"]=find_average(df_leaf_content_1.loc["sucrose"])/ \
model.metabolites.get_by_id("SUCROSE_c").formula_weight

biomass.at["Starch_b","leaf"]=df_leaf_content_4["Starch from Lunn and Hatch 1995 & Ogbaga et al 2016"][0]/ \
                              model.metabolites.get_by_id("STARCH_p").formula_weight

biomass.at["GLC_c","leaf"]=find_average(df_leaf_content_1.loc["glucose"])/ \
model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["FRU_c","leaf"]=find_average(df_leaf_content_1.loc["fructose"])/ \
model.metabolites.get_by_id("FRU_c").formula_weight

starch_mass2 = df_leaf_content_2["Starch (mg/gDW)"][0]
sucrose_mass2 = df_leaf_content_2["Soluble sugar (mg/gDW)"][0]
biomass.at["Cellulose_b","leaf"]=df_leaf_content_2["Cellulose (mg/gDW)"]*(starch_mass2/sucrose_mass2)/ \
                                 model.metabolites.get_by_id("STARCH_p").formula_weight
biomass.at["COUMARATE_c","leaf"]=df_leaf_content_2["Cellulose (mg/gDW)"]/model.metabolites.get_by_id("COUMARATE_c").formula_weight

biomass.at["DIACYLGLYCEROL_p","leaf"]=df_leaf_content_3["DAG (mmol/gDW)"][0]
biomass.at["PHOSPHATIDYL_CHOLINE_r","leaf"]=df_leaf_content_3["PC (mmol/gDW)"][0]
biomass.at["D_Galactosyl_12_diacyl_glycerols_p","leaf"]=df_leaf_content_3["DGDG (mmol/gDW)"][0]
biomass.at["2_Lysophosphatidylcholines_r","leaf"]=df_leaf_content_3["LPC (mmol/gDW)"][0]
biomass.at["Galactosyl_galactosyl_diacyl_glycerols_p","leaf"]=df_leaf_content_3["MGDG (mmol/gDW)"][0]
biomass.at["Triacylglycerols_p","leaf"]=df_leaf_content_3["TAG (mmol/gDW)"][0]
biomass.at["PALMITATE_p","leaf"]=df_leaf_content_5_proc["nmol/mgDW"]["C16:0"]
biomass.at["CPD_9245_p","leaf"]=df_leaf_content_5_proc["nmol/mgDW"]["C16:1"]
biomass.at["CPD_17412_p","leaf"]=df_leaf_content_5_proc["nmol/mgDW"]["C16:2"]
biomass.at["STEARIC_ACID_p","leaf"]=df_leaf_content_5_proc["nmol/mgDW"]["C18:0"]
biomass.at["OLEATE_CPD_p","leaf"]=df_leaf_content_5_proc["nmol/mgDW"]["C18:1"]
biomass.at["Octadecadienoate_p","leaf"]=df_leaf_content_5_proc["nmol/mgDW"]["C18:2"]
biomass.at["LINOLENIC_ACID_p","leaf"]=df_leaf_content_5_proc["nmol/mgDW"]["C18:3"]
biomass.at["ARACHIDIC_ACID_p","leaf"]=df_leaf_content_5_proc["nmol/mgDW"]["C20:0"]
biomass.at["CPD_16709_p","leaf"]=df_leaf_content_5_proc["nmol/mgDW"]["C20:1"]

biomass.at["sASP_b","leaf"]=find_average(df_leaf_content_1.loc["ASP"])/model.metabolites.get_by_id("L_ASPARTATE_c").formula_weight
biomass.at["sSER_b","leaf"]=find_average(df_leaf_content_1.loc["SER"])/model.metabolites.get_by_id("SER_c").formula_weight
biomass.at["THR_c","leaf"]=find_average(df_leaf_content_1.loc["THR"])/model.metabolites.get_by_id("THR_c").formula_weight
biomass.at["ASN_c","leaf"]=find_average(df_leaf_content_1.loc["ASN"])/model.metabolites.get_by_id("ASN_c").formula_weight
biomass.at["PHE_c","leaf"]=find_average(df_leaf_content_1.loc["PHE"])/model.metabolites.get_by_id("PHE_c").formula_weight
biomass.at["TYR_c","leaf"]=find_average(df_leaf_content_1.loc["TYR"])/model.metabolites.get_by_id("TYR_c").formula_weight


### Stem



Extracted sucrose, glucose and fructose content from Njokweni 2015, Figure 3.1 and Figure 3.2

<img src="references/Screenshot_20190715_165703.png" height=50% width=50%>

<img src="references/Screenshot_20190715_170255.png" height=50% width=50%>

In [15]:
df_stem_content_1 = pd.DataFrame(data={"strain":["ICSB338","ICSB73","ICSV213","S35"],
                                       "Sucrose (g/L)":[10.6730769230769,17.7884615384615,
                                                        9.18269230769231,0.048076923076926],
                                       "Glucose (g/L)":[8.13186813186791,2.8571428571426,
                                                        12.2527472527471,0.494505494505204],
                                       "Fructose (g/L)":[63.22097378277,21.6479400749044,
                                                         89.0636704119839,0],
                                       "Starch (g/L)":[11.2911392405063,6.1873417721519,
                                                       5.36708860759494,4.91139240506329]})
df_stem_content_1.set_index("strain",drop=True,inplace=True)
df_stem_content_1

,Sucrose (g/L),Glucose (g/L),Fructose (g/L),Starch (g/L)
strain,,,,
ICSB338,10.673077,8.131868,63.220974,11.291139
ICSB73,17.788462,2.857143,21.647940,6.187342
ICSV213,9.182692,12.252747,89.063670,5.367089
S35,0.048077,0.494505,0.000000,4.911392


Extracted sucrose, starch, glucose, fructose, protein, arabinan, galactan, lignin, xylan and cellulose content of -6 DPA stem from McKinley et al 2016 Figure 3

<img src="references/Screenshot_20190715_170731.png" height=75% width=75%>

In [16]:
df_stem_content_2 = pd.DataFrame(data={"Metabolites":["sucrose","starch","glucose","fructose","protein",
                                                      "arabinan","galactan","lignin","xylan","cellulose"],
                                       "-6 DPA (g/100gDW)":[3.65614798694232,2.48787426500548,8.29162132752991,6.98585418933624,
                                                 1.63220892274211,1.17519042437434,0.457018498367745,11.8824809575626,
                                                 14.8204570184984,28.8030467899891]})
df_stem_content_2.set_index("Metabolites",drop=True,inplace=True)
df_stem_content_2["-6 DPA (mg/gDW)"]=df_stem_content_2["-6 DPA (g/100gDW)"]*10
df_stem_content_2

,-6 DPA (g/100gDW),-6 DPA (mg/gDW)
Metabolites,,
sucrose,3.656148,36.561480
starch,2.487874,24.878743
glucose,8.291621,82.916213
fructose,6.985854,69.858542
protein,1.632209,16.322089
arabinan,1.175190,11.751904
galactan,0.457018,4.570185
lignin,11.882481,118.824810
xylan,14.820457,148.204570


Retrieved average soluble sugar, starch, hemicellulose, cellulose and lignin content from Arai-Sanoh et al 2011, Table 2

In [17]:
df_stem_content_3 = pd.DataFrame(data={"soluble sugar (mg/gDW)":[256],"starch (mg/gDW)":[28],
                                       "hemicellulose (mg/gDW)":[164],"cellulose (mg/gDW)":[305],
                                       "lignin (mg/gDW)":[128]})
df_stem_content_3

,soluble sugar (mg/gDW),starch (mg/gDW),hemicellulose (mg/gDW),cellulose (mg/gDW),lignin (mg/gDW)
0,256,28,164,305,128


Gathered soluble sugar, starch, cellulose, hemicellulose and lignin content from Wang et al 2013, Table 2

In [18]:
df_stem_content_4 = pd.DataFrame(data={"Soluble sugar (mg/gDW)":[157.0],"Starch (mg/gDW)":[9.8],
                                       "Cellulose (mg/gDW)":[281.8],"Hemicellulose (mg/gDW)":[251.3],
                                       "Lignin (mg/gDW)":[32.5]})
df_stem_content_4

,Soluble sugar (mg/gDW),Starch (mg/gDW),Cellulose (mg/gDW),Hemicellulose (mg/gDW),Lignin (mg/gDW)
0,157.0,9.8,281.8,251.3,32.5


Gathered sucrose, fructose, glucose, amino acids, organic acids, trehalose, maltose, mannitol, mannose, galactose and raffinose relative abundance from Njokweni 2015 table 2.1

In [19]:
df_stem_content_5 = pd.DataFrame(data={"strain":["ICSB338","ICSB73","ICSV213","S35"],
                                       "Ala":[9.02,2.55,3.7,1.93],
                                       "Val":[0.34,0,12.6,1.25],
                                       "Ser":[1.46,0.26,1.81,1.45],
                                       "Pro":[0.16,0.14,1.01,0.7],
                                       "Gln":[0.25,0,0.16,0.23],
                                       "Leu":[8.7,4.8,49.4,25],
                                       "Gly":[0.61,0.24,0.2,0.15],
                                       "Asp":[1.45,0.34,1.57,1.88],
                                       "Malate":[7.28,5.22,7.28,3.78],
                                       "Trehalose":[0,0.89,0.82,0.41],
                                       "maltose":[0,0.61,0,0],
                                       "Mannitol":[3.12,1.72,0.18,0.17],
                                       "Mannose":[0.31,4.02,0.27,1.12],
                                       "Galactose":[2.16,7.05,1.13,0.24],
                                       "Raffinose":[0,0.21,0,0],
                                       "Glucose":[2.6,8.63,1.35,2.17],
                                       "Fructose":[1.87,7.49,1.18,6.7],
                                       "Sucrose":[61.8,64.5,94.1,39.2]})
df_stem_content_5.set_index("strain",drop=True,inplace=True)
df_stem_content_5

,Ala,Val,Ser,Pro,Gln,Leu,Gly,Asp,Malate,Trehalose,maltose,Mannitol,Mannose,Galactose,Raffinose,Glucose,Fructose,Sucrose
strain,,,,,,,,,,,,,,,,,,
ICSB338,9.02,0.34,1.46,0.16,0.25,8.7,0.61,1.45,7.28,0.00,0.00,3.12,0.31,2.16,0.00,2.60,1.87,61.8
ICSB73,2.55,0.00,0.26,0.14,0.00,4.8,0.24,0.34,5.22,0.89,0.61,1.72,4.02,7.05,0.21,8.63,7.49,64.5
ICSV213,3.70,12.60,1.81,1.01,0.16,49.4,0.20,1.57,7.28,0.82,0.00,0.18,0.27,1.13,0.00,1.35,1.18,94.1
S35,1.93,1.25,1.45,0.70,0.23,25.0,0.15,1.88,3.78,0.41,0.00,0.17,1.12,0.24,0.00,2.17,6.70,39.2


In [20]:
biomass.at["sSUCROSE_b","stem"] =  df_stem_content_2.at["sucrose","-6 DPA (mg/gDW)"]/ \
                                       model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","stem"]=df_stem_content_2.at["glucose","-6 DPA (mg/gDW)"]/ \
                                       model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["FRU_c","stem"]=df_stem_content_2.at["fructose","-6 DPA (mg/gDW)"]/ \
                                       model.metabolites.get_by_id("FRU_c").formula_weight
biomass.at["Cellulose_b","stem"]=df_stem_content_2.at["cellulose","-6 DPA (mg/gDW)"]/ \
                                       model.metabolites.get_by_id("STARCH_p").formula_weight
biomass.at["COUMARATE_c","stem"]=df_stem_content_2.at["lignin","-6 DPA (mg/gDW)"]/ \
                                       model.metabolites.get_by_id("COUMARATE_c").formula_weight
biomass.at["Xylan_b","stem"]=df_stem_content_2.at["lignin","-6 DPA (mg/gDW)"]/ \
                                       model.metabolites.get_by_id("Xylan_b").formula_weight
#scaling=find_average(df_stem_content_5["Malate"])/find_average(df_stem_content_5["Glucose"])
#biomass.at["sMAL_b","stem"] =scaling*df_stem_content_2.at["glucose","-6 DPA (mg/gDW)"]/ \
#                                       model.metabolites.get_by_id("MAL_c").formula_weight
#
#scaling=find_average(df_stem_content_5["Asp"])/find_average(df_stem_content_5["Glucose"])
#biomass.at["sASP_b","stem"]=scaling*df_stem_content_2.at["glucose","-6 DPA (mg/gDW)"]/ \
#                                       model.metabolites.get_by_id("L_ASPARTATE_c").formula_weight
#
#scaling=find_average(df_stem_content_5["Ser"])/find_average(df_stem_content_5["Glucose"])
#biomass.at["sSER_b","stem"]=scaling*df_stem_content_2.at["glucose","-6 DPA (mg/gDW)"]/ \
#                                       model.metabolites.get_by_id("SER_c").formula_weight
#
#scaling=find_average(df_stem_content_5["Gln"])/find_average(df_stem_content_5["Glucose"])
#biomass.at["sGLN_b","stem"]=scaling*df_stem_content_2.at["glucose","-6 DPA (mg/gDW)"]/ \
#                                       model.metabolites.get_by_id("GLN_c").formula_weight
#
#scaling=find_average(df_stem_content_5["Gly"])/find_average(df_stem_content_5["Glucose"])
#biomass.at["GLY_c","stem"]=scaling*df_stem_content_2.at["glucose","-6 DPA (mg/gDW)"]/ \
#                                       model.metabolites.get_by_id("PRO_c").formula_weight
#
#scaling=find_average(df_stem_content_5["Ala"])/find_average(df_stem_content_5["Glucose"])    
#biomass.at["sALA_b","stem"]=scaling*df_stem_content_2.at["glucose","-6 DPA (mg/gDW)"]/ \
#                                       model.metabolites.get_by_id("L_ALPHA_ALANINE_c").formula_weight
#
#scaling=find_average(df_stem_content_5["Val"])/find_average(df_stem_content_5["Glucose"])    
#biomass.at["VAL_c","stem"]=scaling*df_stem_content_2.at["glucose","-6 DPA (mg/gDW)"]/ \
#                                       model.metabolites.get_by_id("VAL_c").formula_weight
#
#scaling=find_average(df_stem_content_5["Leu"])/find_average(df_stem_content_5["Glucose"])    
#biomass.at["LEU_c","stem"]=scaling*df_stem_content_2.at["glucose","-6 DPA (mg/gDW)"]/ \
#                                       model.metabolites.get_by_id("LEU_c").formula_weight

### Summary

- sucrose, glucose and fructose content from Njokweni 2015 are in g/L. Conversion to mg/gDW is not available and hence these data points are ignored

Assumptions

- Although r

### Root

Retrieve sugar, amino acid and organic acid content from Cambria et al 1983, Table 1

In [21]:
df_root_content_1 = pd.DataFrame(data={"Variety":["Al-sensitive","Al-tolerant"],
                                       "Sugars (mg/gFW)":[4.50,7.50],
                                       "Amino acids (umoles/gFW)":[7.40,7.10],
                                       "organic acids (ueq/gFW)":[16.3,23.64]})
df_root_content_1.set_index("Variety",drop=True,inplace=True)
df_root_content_1

,Sugars (mg/gFW),Amino acids (umoles/gFW),organic acids (ueq/gFW)
Variety,,,
Al-sensitive,4.5,7.4,16.30
Al-tolerant,7.5,7.1,23.64


In [22]:
find_average(df_root_content_1["Amino acids (umoles/gFW)"])

7.25

Retrieve organic acid content from Cambria et al 1983, Table 2

In [23]:
df_root_content_2 = pd.DataFrame(data={"Variety":["Al-sensitive","Al-tolerant"],
                                       "Succinic acid":[0.24,0.17],
                                       "Fumaric acid":[1.44,1.70],
                                       "Malic acid":[3.60,6.53],
                                       "t-Aconitic acid":[2.40,9.50],
                                       "Quinic acid":[0.04,0.17],
                                       "Citric acid":[0.50,0.61],
                                       "Total":[8.22,18.68]})
df_root_content_2.set_index("Variety",drop=True,inplace=True)
df_root_content_2

,Succinic acid,Fumaric acid,Malic acid,t-Aconitic acid,Quinic acid,Citric acid,Total
Variety,,,,,,,
Al-sensitive,0.24,1.44,3.60,2.4,0.04,0.50,8.22
Al-tolerant,0.17,1.70,6.53,9.5,0.17,0.61,18.68


In [24]:
df_root_content_proc = df_root_content_1[df_root_content_1.columns[:2]]
for col in df_root_content_2.columns:
    df_root_content_proc[col+" (umoles/gFW)"] = df_root_content_1["organic acids (ueq/gFW)"]*df_root_content_2[col]/df_root_content_2["Total"]
df_root_content_proc

,Sugars (mg/gFW),Amino acids (umoles/gFW),Succinic acid (umoles/gFW),Fumaric acid (umoles/gFW),Malic acid (umoles/gFW),t-Aconitic acid (umoles/gFW),Quinic acid (umoles/gFW),Citric acid (umoles/gFW),Total (umoles/gFW)
Variety,,,,,,,,,
Al-sensitive,4.5,7.4,0.475912,2.855474,7.138686,4.759124,0.079319,0.991484,16.30
Al-tolerant,7.5,7.1,0.215139,2.151392,8.263876,12.022484,0.215139,0.771970,23.64


In [25]:
biomass.at["sSUCROSE_b","root"] = find_average(df_root_content_proc["Sugars (mg/gFW)"])/model.metabolites.get_by_id("sSUCROSE_b").formula_weight
biomass.at["sGLU_b","root"]= find_average(df_root_content_proc["Amino acids (umoles/gFW)"])
biomass.at["SUC_c","root"]= find_average(df_root_content_proc["Succinic acid (umoles/gFW)"])
biomass.at["FUM_c","root"]= find_average(df_root_content_proc["Fumaric acid (umoles/gFW)"])
biomass.at["MAL_c","root"]= find_average(df_root_content_proc["Malic acid (umoles/gFW)"])
biomass.at["CIS_ACONITATE_c","root"]= find_average(df_root_content_proc["t-Aconitic acid (umoles/gFW)"])
biomass.at["CIT_c","root"]= find_average(df_root_content_proc["Citric acid (umoles/gFW)"])


### Seed


Retrieve soluble sugar, starch, cellulose, hemicellulose and lignin content from Wang et al 2013 Table 2

In [26]:
df_seed_content_1 = pd.DataFrame(data={"Soluble sugar (mg/gDW)":[52.3],"Starch (mg/gDW)":[395.2],
                                     "Cellulose (mg/gDW)":[43.7],"Hemicellulose (mg/gDW)":[147.2],
                                     "Lignin (mg/gDW)":[8.1]})
df_seed_content_1

,Soluble sugar (mg/gDW),Starch (mg/gDW),Cellulose (mg/gDW),Hemicellulose (mg/gDW),Lignin (mg/gDW)
0,52.3,395.2,43.7,147.2,8.1


Estimated FW/DW ratio from Neucere et al 1980, Table I

In [27]:
Moisture = 12.07
DW = 100 - Moisture
FW_DW_ratio = 100/DW
print(FW_DW_ratio)


1.1372682815876265


Retrieved fructose, glucose, sucrose, maltose and raffinose content from Neucere et al 1980, Table I and IV

In [28]:
df_seed_content_2 = pd.DataFrame(data={"variety":["SC301","NSA740","CK60","TX615","GA615"],
                                       "fructose (g/100gDW)":[1.67,1.12,2.32,1.80,1.06],
                                       "glucose (g/100gDW)":[1.78,0.68,2,2.94,0.87],
                                       "sucrose (g/100gDW)":[0.51,0.30,0.58,0.38,0.30],
                                       "maltose (g/100gDW)":[0.35,0.16,0.23,0.78,0.06],
                                       "raffinose (g/100gDW)":[0.1,0.03,0.03,0.11,0.05],
                                       "protein (g/100gFW)":[11.48,14.32,9.90,11.85,9.75],
                                       "fat (g/100gFW)":[3.03,3.49,3.31,2.66,3.00]})
df_seed_content_2_std = pd.DataFrame(data={"variety":["SC301","NSA740","CK60","TX615","GA615"],
                                           "fructose (mg/gDW)":df_seed_content_2["fructose (g/100gDW)"]*10,
                                           "glucose (mg/gDW)":df_seed_content_2["glucose (g/100gDW)"]*10,
                                           "sucrose (mg/gDW)":df_seed_content_2["sucrose (g/100gDW)"]*10,
                                           "maltose (mg/gDW)":df_seed_content_2["maltose (g/100gDW)"]*10,
                                           "raffinose (mg/gDW)":df_seed_content_2["raffinose (g/100gDW)"]*10,
                                           "protein (mg/gDW)":(df_seed_content_2["protein (g/100gFW)"]*FW_DW_ratio)*10,
                                           "fat (mg/gDW)":(df_seed_content_2["fat (g/100gFW)"]*FW_DW_ratio)*10})

df_seed_content_2_std.set_index("variety",drop=True,inplace=True)
df_seed_content_2_std

,fructose (mg/gDW),glucose (mg/gDW),sucrose (mg/gDW),maltose (mg/gDW),raffinose (mg/gDW),protein (mg/gDW),fat (mg/gDW)
variety,,,,,,,
SC301,16.7,17.8,5.1,3.5,1.0,130.558399,34.459229
NSA740,11.2,6.8,3.0,1.6,0.3,162.856818,39.690663
CK60,23.2,20.0,5.8,2.3,0.3,112.589560,37.643580
TX615,18.0,29.4,3.8,7.8,1.1,134.766291,30.251336
GA615,10.6,8.7,3.0,0.6,0.5,110.883657,34.118048


Gather amino acid composition from Mosse, Huet and Baudet 1988

In [29]:
df_seed_content_3 = pd.DataFrame(data={"variety":range(1,13),
                                       "N content (g/100gDW)":[1.51,1.67,1.74,1.86,1.86,1.88,
                                                               2.06,2.25,2.34,2.66,2.86,2.97],
                                       "Gly (g/16g N)":[3.4,3.15,3.05,3.05,3.1,2.9,2.85,2.8,3,2.75,2.55,2.5],
                                       "Ala (g/16g N)":[9.1,9.2,9.2,9.4,9.5,9.1,9.6,9.5,9.3,9.8,9.7,10],
                                       "Val (g/16g N)":[5.2,5.5,5.3,5.6,5.6,5.2,5.6,5.3,5.3,5.3,5.2,5.1],
                                       "Leu (g/16g N)":[12.7,13.9,13.6,13.8,14.2,13.6,
                                                        14.4,13.8,13.3,14.2,14.1,14.7],
                                       "Ile (g/16g N)":[4.1,4.3,4.2,4.3,4.4,4.1,4.3,4.2,4.2,4.4,4.3,4.4],
                                       "Ser (g/16g N)":[4.7,4.6,4.5,4.6,4.7,4.5,4.5,4.7,4.6,4.7,4.5,4.5],
                                       "Thr (g/16g N)":[3.35,3.25,3.15,3.3,3.3,3.2,3.15,3.2,3.2,3.2,3.1,3],
                                       "Tyr (g/16g N)":[3.9,4.2,4.1,4.2,4.3,3.9,4.3,3.9,4,4,4,4.1],
                                       "Phe (g/16g N)":[5.2,5.5,5.4,5.5,5.5,5.2,5.4,5.5,5.3,5.5,5.6,5.7],
                                       "Trp (g/16g N)":[1.08,1.03,1,1.01,1.11,0.84,1,1.05,1.05,1.02,1,1],
                                       "Pro (g/16g N)":[8.1,8.8,8.5,8.6,8.9,8.2,8.9,8.3,8.1,8.4,8.3,8.4],
                                       "Met (g/16g N)":[2,1.65,1.55,1.6,1.7,1.35,1.55,1.35,1.65,1.6,1.4,1.4],
                                       "Half Cys (g/16g N)":[2.15,1.9,1.7,1.75,1.9,1.75,
                                                             1.65,1.85,1.9,1.6,1.45,1.5],
                                       "Lys (g/16g N)":[2.5,2.3,2.3,2.3,2.3,2.1,2.15,2.05,2.3,1.95,1.95,1.65],
                                       "His (g/16g N)":[2.3,2.2,2.15,2.25,2.3,2.2,2.15,2.3,2.1,2.05,2,2],
                                       "Arg (g/16g N)":[4.1,4,4,4,4.1,3.8,3.8,3.7,4.2,3.7,3.7,3.1],
                                       "Asp (g/100gDW)":[6.7,7,7,7,6.9,9.7,7,7.7,7.3,7.3,7.1,6.7],
                                       "Glt (g/100gDW)":[20.3,21,20.8,21.4,21.7,20.8,
                                                         22,22.4,21.2,22.6,22.3,22.9]})
df_seed_content_3_std = pd.DataFrame(data={"variety":range(1,13),
                                       "Gly (mg/gDW)":(df_seed_content_3["Gly (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Ala (mg/gDW)":(df_seed_content_3["Ala (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Val (mg/gDW)":(df_seed_content_3["Val (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Leu (mg/gDW)":(df_seed_content_3["Leu (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Ile (mg/gDW)":(df_seed_content_3["Ile (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Ser (mg/gDW)":(df_seed_content_3["Ser (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Thr (mg/gDW)":(df_seed_content_3["Thr (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Tyr (mg/gDW)":(df_seed_content_3["Tyr (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Phe (mg/gDW)":(df_seed_content_3["Phe (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Trp (mg/gDW)":(df_seed_content_3["Trp (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Pro (mg/gDW)":(df_seed_content_3["Pro (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Met (mg/gDW)":(df_seed_content_3["Met (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Cys (mg/gDW)":0.5*(df_seed_content_3["Half Cys (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Lys (mg/gDW)":(df_seed_content_3["Lys (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "His (mg/gDW)":(df_seed_content_3["His (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Arg (mg/gDW)":(df_seed_content_3["Arg (g/16g N)"]/16)* \
                                                            df_seed_content_3["N content (g/100gDW)"]*10,
                                       "Asp (mg/gDW)":df_seed_content_3["Asp (g/100gDW)"]*10,
                                       "Glt (mg/gDW)":df_seed_content_3["Glt (g/100gDW)"]*10})
df_seed_content_3_std.set_index("variety",drop=True,inplace=True)
df_seed_content_3_std

,Gly (mg/gDW),Ala (mg/gDW),Val (mg/gDW),Leu (mg/gDW),Ile (mg/gDW),Ser (mg/gDW),Thr (mg/gDW),Tyr (mg/gDW),Phe (mg/gDW),Trp (mg/gDW),Pro (mg/gDW),Met (mg/gDW),Cys (mg/gDW),Lys (mg/gDW),His (mg/gDW),Arg (mg/gDW),Asp (mg/gDW),Glt (mg/gDW)
variety,,,,,,,,,,,,,,,,,,
1,3.208750,8.588125,4.907500,11.985625,3.869375,4.435625,3.161563,3.680625,4.907500,1.019250,7.644375,1.887500,1.014531,2.359375,2.170625,3.869375,67.0,203.0
2,3.287812,9.602500,5.740625,14.508125,4.488125,4.801250,3.392188,4.383750,5.740625,1.075063,9.185000,1.722187,0.991562,2.400625,2.296250,4.175000,70.0,210.0
3,3.316875,10.005000,5.763750,14.790000,4.567500,4.893750,3.425625,4.458750,5.872500,1.087500,9.243750,1.685625,0.924375,2.501250,2.338125,4.350000,70.0,208.0
4,3.545625,10.927500,6.510000,16.042500,4.998750,5.347500,3.836250,4.882500,6.393750,1.174125,9.997500,1.860000,1.017188,2.673750,2.615625,4.650000,70.0,214.0
5,3.603750,11.043750,6.510000,16.507500,5.115000,5.463750,3.836250,4.998750,6.393750,1.290375,10.346250,1.976250,1.104375,2.673750,2.673750,4.766250,69.0,217.0
6,3.407500,10.692500,6.110000,15.980000,4.817500,5.287500,3.760000,4.582500,6.110000,0.987000,9.635000,1.586250,1.028125,2.467500,2.585000,4.465000,97.0,208.0
7,3.669375,12.360000,7.210000,18.540000,5.536250,5.793750,4.055625,5.536250,6.952500,1.287500,11.458750,1.995625,1.062188,2.768125,2.768125,4.892500,70.0,220.0
8,3.937500,13.359375,7.453125,19.406250,5.906250,6.609375,4.500000,5.484375,7.734375,1.476563,11.671875,1.898438,1.300781,2.882812,3.234375,5.203125,77.0,224.0
9,4.387500,13.601250,7.751250,19.451250,6.142500,6.727500,4.680000,5.850000,7.751250,1.535625,11.846250,2.413125,1.389375,3.363750,3.071250,6.142500,73.0,212.0


In [30]:
biomass.at["COUMARATE_c","seed"] = df_seed_content_1["Lignin (mg/gDW)"]/ \
                                            model.metabolites.get_by_id("COUMARATE_c").formula_weight
biomass.at["sSUCROSE_b","seed"] = find_average(df_seed_content_2_std["sucrose (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","seed"]=find_average(df_seed_content_2_std["glucose (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["FRU_c","seed"]=find_average(df_seed_content_2_std["fructose (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("FRU_c").formula_weight
biomass.at["L_PHOSPHATIDATE_p","leaf"]=14.2253521126762/model.metabolites.get_by_id("L_PHOSPHATIDATE_p").formula_weight
biomass.at["Protein_b","leaf"]=8.339/proteinMW

biomass.at["ARG_c","seed"] = find_average(df_seed_content_3_std["Arg (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("ARG_c").formula_weight
biomass.at["HIS_c","seed"] = find_average(df_seed_content_3_std["His (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("HIS_c").formula_weight
biomass.at["LYS_c","seed"] = find_average(df_seed_content_3_std["Lys (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("LYS_c").formula_weight
biomass.at["sASP_b","seed"] = find_average(df_seed_content_3_std["Asp (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("L_ASPARTATE_c").formula_weight
biomass.at["sGLU_b","seed"] = find_average(df_seed_content_3_std["Glt (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("sGLU_b").formula_weight
biomass.at["sSER_b","seed"] = find_average(df_seed_content_3_std["Ser (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("sSER_b").formula_weight
biomass.at["THR_c","seed"] = find_average(df_seed_content_3_std["Thr (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("THR_c").formula_weight
biomass.at["CYS_c","seed"] = find_average(df_seed_content_3_std["Cys (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("CYS_c").formula_weight
biomass.at["GLY_c","seed"] = find_average(df_seed_content_3_std["Gly (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("GLY_c").formula_weight
biomass.at["PRO_c","seed"] = find_average(df_seed_content_3_std["Pro (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("PRO_c").formula_weight
biomass.at["sALA_b","seed"] = find_average(df_seed_content_3_std["Ala (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("L_ALPHA_ALANINE_c").formula_weight
biomass.at["VAL_c","seed"] = find_average(df_seed_content_3_std["Val (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("VAL_c").formula_weight
biomass.at["ILE_c","seed"] = find_average(df_seed_content_3_std["Ile (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("ILE_c").formula_weight
biomass.at["LEU_c","seed"] = find_average(df_seed_content_3_std["Leu (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("LEU_c").formula_weight
biomass.at["MET_c","seed"] = find_average(df_seed_content_3_std["Met (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("MET_c").formula_weight
biomass.at["PHE_c","seed"] = find_average(df_seed_content_3_std["Phe (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("PHE_c").formula_weight
biomass.at["TYR_c","seed"] = find_average(df_seed_content_3_std["Tyr (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("TYR_c").formula_weight
biomass.at["TRP_c","seed"] = find_average(df_seed_content_3_std["Trp (mg/gDW)"])/ \
                                            model.metabolites.get_by_id("TRP_c").formula_weight
biomass.to_csv("Data/biomass_sorghum.csv")

In [31]:
biomass

,type,leaf,stem,root,seed
,,,,,
sSUCROSE_b,,0.015871,0.106812,0.017529,0.012095
GLC_c,,0.218131,0.460247,0.000000,0.091809
FRU_c,,0.001114,0.387767,0.000000,0.088479
Starch_b,,0.158102,0.000000,0.000000,0.000000
Cellulose_b,,0.235781,1.776424,0.000000,0.000000
...,...,...,...,...,...
pLEU_b,protein,0.000000,0.000000,0.000000,0.000000
pPRO_b,protein,0.000000,0.000000,0.000000,0.000000
pCYS_b,protein,0.000000,0.000000,0.000000,0.000000


### References

- Wang, J.S., Wang, M.L., Spiertz, J.H.J., Liu, Z.X., Han, L.P. and Xie, G.H. (2013) Genetic variation in yield and chemical composition of wide range of sorghum accessions grown in north-west China. Res. Crop., 14, 95–105.
- Ogbaga, C.C., Stepien, P., Dyson, B.C., Rattray, N.J.W., Ellis, D.I., Goodacre, R. and Johnson, G.N. (2016) Biochemical analyses of sorghum varieties reveal differential responses to drought J.-S. Zhang, ed. PLoS One, 11, e0154423. https://doi.org/10.1371/journal.pone.0154423.
- Vanhercke, T., Belide, S., Taylor, M.C., et al. (2019) Up-regulation of lipid biosynthesis increases the oil content in leaves of Sorghum bicolor. Plant Biotechnol. J., 17, 220–232. Available at: http://doi.wiley.com/10.1111/pbi.12959.
- Lunn, J.E. and Hatch, M.D. (1995) Primary partitioning and storage of photosynthate in sucrose and starch in leaves of C4 plants. Planta, 197, 5000. https://doi.org/10.1007/BF00202661.
- Njokweni, A.P. (University of the W.C. (2015) COMPARATIVE ANALYSIS OF SUGAR-BIOSYNTHESIS PROTEINS OF SORGHUM STEMS AND THE INVESTIGATION OF THEIR ROLE IN HYPEROSMOTIC STRESS TOLERANCE. University of the Western Cape. 
https://pdfs.semanticscholar.org/cb3f/6b1c4f9b2acae9fe0afba1c3603c32fdb2c7.pdf.
- Arai-Sanoh, Y., Ida, M., Zhao, R., et al. (2011) Genotypic variations in non-structural carbohydrate and cell-wall components of the stem in rice, sorghum, and sugar vane. Biosci. Biotechnol. Biochem., 75, 1104–12. Available at: http://www.tandfonline.com/doi/full/10.1271/bbb.110009.
- Cambria, J., Galvani, F.R., Estevao, M.M. and Sant’Anna, R. (1983) Effects of Aluminum on Organic Acid,’ Sugar and Amino Acid Composition of the Root System of Sorghum (Sorghum Bicolor L. Moench). J. Plant Nutr., 6, 313–322. https://doi.org/10.1080/01904168309363092.
- Neucere, N.J. and Sumrell, G. (1980) Chemical composition of different varieties of grain sorghum. J. Agric. Food Chem., 28, 19–21. http://doi.org/10.1021/jf60227a022.